In [2]:
#coding:utf-8
import numpy as np
import json
# from model import ConceptFlow, use_cuda
# from preprocession import prepare_data, build_vocab, gen_batched_data
import torch
import warnings
import yaml
import os
import requests
import re
import random
import gensim.downloader as api
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import utils as nn_utils
warnings.filterwarnings('ignore')

In [3]:
w2v_model = api.load('word2vec-google-news-300')

In [4]:
data_dir = "D:\\13.PURDUE\\Data\\ConceptFlow(ECCF)_data"
with open('%s/resource.txt' % data_dir) as f:
    d = json.loads(f.readline())

    csk_triples = d['csk_triples']
    csk_entities = d['csk_entities']
    raw_vocab = d['vocab_dict']
    kb_dict = d['dict_csk']
    dict_csk_entities = d['dict_csk_entities']
    dict_csk_triples = d['dict_csk_triples']

# create_all_entities_one_hop & create_all_triples_one_hop

In [188]:
csk_triples = csk_triples
dict_csk_entities = dict_csk_entities
csk_entities = csk_entities
res = re.findall( r'\w+|[^\s\w]+', post2)
post_entities = []

tmp_e = ''
tmp_o = ''

for i in res:
    if i in csk_entities:
        post_entities.append(i)
    else:
        continue
print('post_entities',post_entities)


# find one_hop_triples from post entities
all_triples_one_hop = []

# find one_hop_entities from post entities
tmp_id_one_hop = []

threshold = 0.
for entity in post_entities:
    for e_o in csk_triples:
        e_o_split = e_o.split(',')
        tmp_e = e_o_split[0]
        tmp_o = e_o_split[2].lstrip()
        tmp_pair = [tmp_e, tmp_o]
        if entity in tmp_pair:
            if entity == tmp_pair[0] and tmp_pair[1] not in post_entities:
                if entity in w2v_model.key_to_index and tmp_pair[1] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score = w2v_model.similarity(entity,tmp_pair[1])
                    all_triples_one_hop.append(dict_csk_triples[e_o])
                    if tmp_pair[1] in dict_csk_entities and sim_score>threshold:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[1]])                    
                else:
                    all_triples_one_hop.append(dict_csk_triples[e_o])
                    if tmp_pair[1] in dict_csk_entities:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[1]])
                        
            elif entity == tmp_pair[1] and tmp_pair[0] not in post_entities:
                if entity in w2v_model.key_to_index and tmp_pair[0] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score = w2v_model.similarity(entity,tmp_pair[0])
                    if tmp_pair[0] in dict_csk_entities and sim_score>threshold:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[0]])
                        all_triples_one_hop.append(dict_csk_triples[e_o])
                else:
                    if tmp_pair[0] in dict_csk_entities:
                        all_triples_one_hop.append(dict_csk_triples[e_o])
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[0]])

            else:
                continue
        else:
            continue

id_one_hop = [i for n, i in enumerate(tmp_id_one_hop) if i not in tmp_id_one_hop[:n]]

# print(id_one_hop)
print(len(id_one_hop))
print(id_one_hop)
print(len(all_triples_one_hop))

post_entities ['comet', 'sky', 'felt', 'cowboys', 'shoot', 'celestial', 'shooters']
6
[2184, 2159, 6039, 7737, 14659, 1376]
135


In [5]:
csk_triples = d['csk_triples']
csk_entities = d['csk_entities']
raw_vocab = d['vocab_dict']
kb_dict = d['dict_csk']
dict_csk_entities = d['dict_csk_entities']
dict_csk_triples = d['dict_csk_triples']

In [6]:
post = "I wear glasses and at the age of seventeen I would have been mortified to be seen in a photo wearing them."
post2 = "As the comet streaked across the Chicago sky, I felt as though time had looped back to the Wild West, where cowboys would shoot at such celestial wonders with their six-shooters."

# Checking similarity between entity and one-hop entity

In [7]:
csk_triples = csk_triples
dict_csk_entities = dict_csk_entities
csk_entities = csk_entities
res = re.findall( r'\w+|[^\s\w]+', post2)


post_entities = []
one_hop_triples_id = []
only_two = []
one_two_triple = []
tmp_one_two_triple = []
tmp_id_one_hop = []
tmp_e = ''
tmp_o = ''

for i in res:
    if i in csk_entities:
        post_entities.append(i)
    else:
        continue
print('post_entities',post_entities)

threshold_one_hop_triples = 0.4
threshold_two_hop_triples = 0.4
for entity in post_entities:
    kb_d = kb_dict[entity]
    for e_o in kb_d:
        e_o_split = e_o.split(',')
        tmp_e = e_o_split[0]
        tmp_o = e_o_split[2].lstrip()
        tmp_pair = [tmp_e, tmp_o]
        if entity == tmp_pair[0] and tmp_pair[1] not in post_entities:
            tmp_id_one_hop.append(dict_csk_entities[tmp_pair[1]])
            one_hop_triples_id.append(dict_csk_triples[e_o])
            if tmp_pair[1] in kb_dict:
                tmp_pair1_kb_d = kb_dict[tmp_pair[1]]
                for tmp_pair1_e_o in tmp_pair1_kb_d:
                    tmp_pair1_e_o_split = tmp_pair1_e_o.split(',')
                    tmp_pair1_e = tmp_pair1_e_o_split[0]
                    tmp_pair1_o = tmp_pair1_e_o_split[2].lstrip()
                    if tmp_pair1_e in w2v_model.key_to_index and tmp_pair1_o in w2v_model.key_to_index:
                        sim_score = w2v_model.similarity(tmp_pair1_e,tmp_pair1_o)
                        sim_score2 = w2v_model.similarity(entity,tmp_pair1_o)
                        if sim_score2 > threshold_one_hop_triples:
                            print('entity:', entity, 'two_hop:', tmp_pair1_o, sim_score2)
                        if sim_score2 < threshold_two_hop_triples:
                            tmp_one_two_triple.append(dict_csk_triples[tmp_pair1_e_o])
                    else:
                        continue
#                     id_two_hop = [i for n, i in enumerate(tmp_one_two_triple) if i not in tmp_one_two_triple[:n]]
                    one_two_triple.append(tmp_one_two_triple)
                tmp_one_two_triple = []
        elif entity == tmp_pair[1] and tmp_pair[0] not in post_entities:
            tmp_id_one_hop.append(dict_csk_entities[tmp_pair[0]])
            one_hop_triples_id.append(dict_csk_triples[e_o])
            if tmp_pair[0] in kb_dict:
                tmp_pair2_kb_d = kb_dict[tmp_pair[0]]
                tmp_one_two_triple = []
                for tmp_pair2_e_o in tmp_pair2_kb_d:
                    tmp_pair2_e_o_split = tmp_pair2_e_o.split(',')
                    tmp_pair2_e = tmp_pair2_e_o_split[0]
                    tmp_pair2_o = tmp_pair2_e_o_split[2].lstrip()
                    if tmp_pair2_e in w2v_model.key_to_index and tmp_pair2_o in w2v_model.key_to_index:
                        sim_score = w2v_model.similarity(tmp_pair2_e,tmp_pair2_o)
                        sim_score2 = w2v_model.similarity(entity,tmp_pair2_o)
                        if sim_score2 < threshold_two_hop_triples:
                            tmp_one_two_triple.append(dict_csk_triples[tmp_pair2_e_o])
                    else:
                        continue
                    one_two_triple.append(tmp_one_two_triple)
                tmp_one_two_triple = []
        else:
            continue

# ☆☆☆☆☆☆☆☆☆☆☆☆ all_entities_one_hop ☆☆☆☆☆☆☆☆☆☆☆☆
id_one_hop = [i for n, i in enumerate(tmp_id_one_hop) if i not in tmp_id_one_hop[:n]]
# print(id_one_hop)
# print('before', len(id_one_hop))
            
# ☆☆☆☆☆☆☆☆☆☆☆☆ all_triples_one_hop ☆☆☆☆☆☆☆☆☆☆☆☆
one_hop_triples_id = [i for n, i in enumerate(one_hop_triples_id) if i not in one_hop_triples_id[:n]]

# print('one_hop_triples_id', len(one_hop_triples_id))
# print(one_hop_triples_id)

one_two_triple = [i for n, i in enumerate(one_two_triple) if i not in one_two_triple[:n]]

# print('two_hop_triples_id', len(one_two_triple))
# print(one_two_triple)


        
    

post_entities ['comet', 'sky', 'felt', 'cowboys', 'shoot', 'celestial', 'shooters']
entity: comet two_hop: earth 0.42217132
entity: comet two_hop: universe 0.45812893
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: comet two_hop: orbit 0.4939087
entity: sky two_hop: moon 0.4920849
entity: sky two_hop: moon 0.4920849
entity: sky two_hop:

# Only_two

In [189]:
tmp_e2 = ''
tmp_o2 = ''
tmp_pair2= []
# find two hope from post entities
lsk = ['loop, RelatedTo, chicago', 'shoot, RelatedTo, comet', 'dumb, RelatedTo, deaf', 'walk, RelatedTo, door', 'pier, RelatedTo, boat']
# id2word = dict()
# for key in word2id.keys():
#     id2word[word2id[key]] = key
swapped_dict = {value: key for key, value in dict_csk_entities.items()}
entities_one_hop = []
for i in id_one_hop:
    entities_one_hop.append(swapped_dict[i])
entities_one_hop2 = entities_one_hop[:5]
threshold2 = 0.05
tmp_entities_two_hop = []

for entity2 in entities_one_hop:
    for e_o2 in csk_triples:
        e_o2 = e_o2.split(',')
        tmp_e2 = e_o2[0]
        tmp_o2 = e_o2[2].lstrip()
        tmp_pair2 = [tmp_e2, tmp_o2]
        if entity2 in tmp_pair2:
            if entity2 == tmp_pair2[0] and tmp_pair2[1] not in post_entities and tmp_pair2[1] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[1] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[1])
                    if tmp_pair2[1] in dict_csk_entities and sim_score2 > threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[1]])                  
                    else:
                        if tmp_pair2[1] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[1]])
            elif entity2 == tmp_pair2[1] and tmp_pair2[0] not in post_entities and tmp_pair2[0] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[0] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[0])
                    if tmp_pair2[0] in dict_csk_entities and sim_score2>threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[0]])                  
                    else: 
                        if tmp_pair2[0] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[0]])
            else:
                continue
        else:
            continue

# print(len(tmp_entities_two_hop))
id_two_hop = [i for n, i in enumerate(tmp_entities_two_hop) if i not in tmp_entities_two_hop[:n]]
print(len(id_two_hop))
print(id_two_hop)
# print(entities_two_hop)

22
[8471, 8676, 16781, 21668, 1675, 1566, 12327, 14148, 84, 22579, 19000, 18183, 8657, 16342, 8525, 688, 12514, 18460, 10493, 17068, 13307, 15144]


# create_post_triples

In [12]:
# Function to create the post_triples list
def create_post_triples(post, csk_entities):
    csk_entities = csk_entities
    post_csk_entities = {}
    count = 1
    res = re.findall( r'\w+|[^\s\w]+', post2)
    print(res)
    print(len(res))
    for i in res:
#         print(i)
        if i in post_csk_entities.keys():
            continue

        elif i in csk_entities:
            post_csk_entities[i] = count
            count = count+1
        else:
            continue
            
    post_triples = []
    for word in res:
        if word in post_csk_entities:
            post_triples.append(post_csk_entities[word])
        else:
            post_triples.append(0)  # Use 0 for words that are not in dict_csk_entities
    return post_triples

In [14]:
post_triples = create_post_triples(post2, csk_entities)

print("Post Triples:", post_triples)

['As', 'the', 'comet', 'streaked', 'across', 'the', 'Chicago', 'sky', ',', 'I', 'felt', 'as', 'though', 'time', 'had', 'looped', 'back', 'to', 'the', 'Wild', 'West', ',', 'where', 'cowboys', 'would', 'shoot', 'at', 'such', 'celestial', 'wonders', 'with', 'their', 'six', '-', 'shooters', '.']
36
Post Triples: [0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 5, 0, 0, 6, 0, 0, 0, 0, 0, 7, 0]


# one_two_triple

# notepad for last one

In [447]:
tmp_e2 = ''
tmp_o2 = ''
tmp_pair2= []
# find two hope from post entities
lsk = ['loop, RelatedTo, chicago', 'shoot, RelatedTo, comet', 'dumb, RelatedTo, deaf', 'walk, RelatedTo, door', 'pier, RelatedTo, boat']
# id2word = dict()
# for key in word2id.keys():
#     id2word[word2id[key]] = key
swapped_dict = {value: key for key, value in dict_csk_entities.items()}
entities_one_hop = []
for i in id_one_hop:
    entities_one_hop.append(swapped_dict[i])
threshold2 = 0.01
tmp_entities_two_hop = []

# ☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆
one_two_triples = []
tmp_one_two_triples = []
# tmp_one_two_triples.append(dict_csk_triples[e_o])
# one_two_triples.append(tmp_one_two_triples)
for entity2 in entities_one_hop:
    tmp_one_two_triples = []
    for e_o2 in csk_triples:
        e_o2_split = e_o2.split(',')
        tmp_e2 = e_o2_split[0]
        tmp_o2 = e_o2_split[2].lstrip()
        tmp_pair2 = [tmp_e2, tmp_o2]
        if entity2 in tmp_pair2:
            if entity2 == tmp_pair2[0] and tmp_pair2[1] not in post_entities and tmp_pair2[1] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[1] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[1])
                    if tmp_pair2[1] in dict_csk_entities and sim_score2 > threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[1]])
                        tmp_one_two_triples.append(dict_csk_triples[e_o2])
                    else:
                        if tmp_pair2[1] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[1]])
#                             tmp_one_two_triples.append(dict_csk_triples[e_o2])
            elif entity2 == tmp_pair2[1] and tmp_pair2[0] not in post_entities and tmp_pair2[0] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[0] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[0])
                    if tmp_pair2[0] in dict_csk_entities and sim_score2>threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[0]])
                        tmp_one_two_triples.append(dict_csk_triples[e_o2])
                    else: 
                        if tmp_pair2[0] in dict_csk_entities:
#                             tmp_one_two_triples.append(dict_csk_triples[e_o2])
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[0]])
            else:
                continue
        else:
            continue
    one_two_triples.append(tmp_one_two_triples)

# print(len(tmp_entities_two_hop))
id_two_hop = [i for n, i in enumerate(tmp_entities_two_hop) if i not in tmp_entities_two_hop[:n]]
print(len(id_two_hop))
print(id_two_hop)
# print(entities_two_hop)
print('len_one_two_triples',len(one_two_triples))
# print(one_two_trizples)

761
[20599, 17692, 14904, 19511, 16090, 15172, 7290, 18329, 20258, 20534, 11730, 13217, 4444, 20110, 13316, 21160, 20085, 6171, 21732, 11817, 2167, 5540, 13169, 13583, 5999, 12258, 8092, 20761, 2899, 1745, 22503, 13278, 3254, 15373, 19732, 18049, 21070, 14568, 9357, 8723, 6558, 20219, 5855, 17877, 5651, 13145, 1052, 14863, 13966, 4489, 11281, 16930, 3839, 15084, 995, 14657, 17183, 19263, 4024, 17124, 10242, 6407, 14154, 2634, 4309, 4729, 14602, 3949, 17575, 22344, 18381, 9214, 2362, 5462, 11687, 19707, 4662, 6404, 20754, 16961, 19547, 4435, 10063, 14665, 6989, 16686, 17108, 6531, 8856, 8410, 880, 6654, 16528, 16806, 8677, 11727, 19213, 589, 720, 1572, 20237, 20925, 1824, 13578, 15799, 16320, 12417, 2704, 6059, 16928, 17142, 16516, 2497, 10441, 12927, 22153, 12276, 6576, 4449, 19600, 15732, 556, 14297, 17456, 5106, 12158, 14890, 18452, 15408, 2381, 13924, 21235, 13175, 6228, 2881, 2236, 1634, 17343, 21628, 5613, 4169, 521, 646, 16402, 7267, 12830, 11947, 2569, 20186, 11012, 1922, 19908,

# Utils

In [6]:
def padding(sent, l):
    return sent + ['_EOS'] + ['_PAD'] * (l-len(sent)-1)

def padding_triple_id(entity2id, triple, num, l):
    newtriple = []
    for i in range(len(triple)):
        for j in range(len(triple[i])):
            for k in range(len(triple[i][j])):
                if triple[i][j][k] in entity2id:
                    triple[i][j][k] = entity2id[triple[i][j][k]]
                else:
                    triple[i][j][k] = entity2id['_NONE']

    for tri in triple:
        newtriple.append(tri + [[entity2id['_PAD_H'], entity2id['_PAD_R'], entity2id['_PAD_T']]] * (l - len(tri)))
    pad_triple = [[entity2id['_PAD_H'], entity2id['_PAD_R'], entity2id['_PAD_T']]] * l
    return newtriple + [pad_triple] * (num - len(newtriple))

def build_kb_adj_mat(kb_adj_mats, fact_dropout):
    """Create sparse matrix representation for batched data"""
    mats0_batch = np.array([], dtype=int)
    mats0_0 = np.array([], dtype=int)
    mats0_1 = np.array([], dtype=int)
    vals0 = np.array([], dtype=float)

    mats1_batch = np.array([], dtype=int)
    mats1_0 = np.array([], dtype=int)
    mats1_1 = np.array([], dtype=int)
    vals1 = np.array([], dtype=float)

    for i in range(kb_adj_mats.shape[0]):
        (mat0_0, mat0_1, val0), (mat1_0, mat1_1, val1) = kb_adj_mats[i]
        assert len(val0) == len(val1)
        num_fact = len(val0)
        num_keep_fact = int(np.floor(num_fact * (1 - fact_dropout)))
        mask_index = np.random.permutation(num_fact)[ : num_keep_fact]
        # mat0
        mats0_batch = np.append(mats0_batch, np.full(len(mask_index), i, dtype=int))
        mats0_0 = np.append(mats0_0, mat0_0[mask_index])
        mats0_1 = np.append(mats0_1, mat0_1[mask_index])
        vals0 = np.append(vals0, val0[mask_index])
        # mat1
        mats1_batch = np.append(mats1_batch, np.full(len(mask_index), i, dtype=int))
        mats1_0 = np.append(mats1_0, mat1_0[mask_index])
        mats1_1 = np.append(mats1_1, mat1_1[mask_index])
        vals1 = np.append(vals1, val1[mask_index])

    return (mats0_batch, mats0_0, mats0_1, vals0), (mats1_batch, mats1_0, mats1_1, vals1)

# How to divide triples into one-hop tirple and two-hop triple

In [223]:
expected_length_response = random.randrange(20,30)
expected_response = [-1] * expected_length_response
# print(expected_length_response)
possible_one_hop_index_range = list(range(3, expected_length_response))
num_of_one_hop = random.randrange(4, 9)
num_of_two_hop = random.randrange(1,3)
possible_hop_index = random.sample(possible_one_hop_index_range, k=num_of_one_hop)
possible_two_index = random.sample(possible_hop_index, k = num_of_two_hop)
# possible_hop_index = np.array(possible_hop_index) - np.array(possible_two_index)
print('possible_hop_index',possible_hop_index)
print('possible_two_index',possible_two_index)


# res = filter(lambda i: i not in possible_hop_index, possible_two_index)
for i in possible_two_index:
    try:
        possible_hop_index.remove(i)
    except ValueError:
        pass
# possible_hop_index = res
print('possible_hop_index',possible_hop_index)
print('possible_two_index',possible_two_index)

# print('possible_one_hop_index',possible_one_hop_index)
replacements = np.random.randint(5, size = num_of_one_hop-num_of_two_hop)
replacements_s = [100] * num_of_two_hop
# print('replacements',replacements)
# print('before', expected_response)
for index in range(len(possible_hop_index)):
    expected_response[possible_hop_index[index]] = replacements[index]
for index in range(len(possible_two_index)):
    expected_response[possible_two_index[index]] = replacements_s[index]
# expected_response[possible_one_hop_index[0]] = 0
# expected_response[possible_one_hop_index[1]] = 0
print('afeter', expected_response)
print(possible_one_hop_index_range)
print(possible_one_hop_index)
# for exp in range(10):
#     expected_length_response = random.randrange(20,30)
#     print(expected_length_response)

possible_hop_index [10, 19, 16, 9, 14]
possible_two_index [9]
possible_hop_index [10, 19, 16, 14]
possible_two_index [9]
afeter [-1, -1, -1, -1, -1, -1, -1, -1, -1, 100, 4, -1, -1, -1, 4, -1, 4, -1, -1, 0]
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[23, 16, 10, 24, 21]


# ★ New post preprocessing

In [192]:
data_dir = "D:\\13.PURDUE\\Data\\ConceptFlow(ECCF)_data"
with open('%s/resource.txt' % data_dir) as f:
    d = json.loads(f.readline())

    csk_triples = d['csk_triples']
    csk_entities = d['csk_entities']
    raw_vocab = d['vocab_dict']
    kb_dict = d['dict_csk']
    dict_csk_entities = d['dict_csk_entities']
    dict_csk_triples = d['dict_csk_triples']

In [231]:
def generate_response_format(post, csk_triples, csk_entities, kb_dict, dict_csk_entities, dict_csk_triples):
    res = re.findall( r'\w+|[^\s\w]+', post)
    count = 1
    # Simulate entity and triple index mapping
#     entity2id = {entity: idx for idx, entity in enumerate(entities)}
    post_csk_entities = {}
    triple_indices = 0  # Assuming each triple gets a unique index
    post_triples = np.zeros(len(res))
    csk_triples = csk_triples
    dict_csk_entities = dict_csk_entities
    csk_entities = csk_entities
    tmp_id_one_hop = []
    post_entities = []
    one_hop_triples_id = []
    tmp_only_two = []
    only_two = []
    one_two_triple = []
    tmp_one_two_triple = []

    tmp_e = ''
    tmp_o = ''
    for i in res:
        if i in csk_entities:
            post_entities.append(i)
        else:
            continue
    print('post_entities',post_entities)
    
    for i in res:
#         print(i)
        if i in post_csk_entities.keys():
            continue

        elif i in csk_entities:
            post_csk_entities[i] = count
            count = count+1
        else:
            continue
            
    post_triples = []
    for word in res:
        if word in post_csk_entities:
            post_triples.append(post_csk_entities[word])
        else:
            post_triples.append(0)  # Use 0 for words that are not in dict_csk_entities
    
    threshold_one_hop_triples = 0.4
    threshold_two_hop_triples = 0.5
    for entity in post_entities:
        kb_d = kb_dict[entity]
        for e_o in kb_d:
            e_o_split = e_o.split(',')
            tmp_e = e_o_split[0]
            tmp_o = e_o_split[2].lstrip()
            tmp_pair = [tmp_e, tmp_o]
            if entity == tmp_pair[0] and tmp_pair[1] not in post_entities:
                tmp_id_one_hop.append(dict_csk_entities[tmp_pair[1]])
                one_hop_triples_id.append(dict_csk_triples[e_o])
                if tmp_pair[1] in kb_dict:
                    tmp_pair1_kb_d = kb_dict[tmp_pair[1]]
                    for tmp_pair1_e_o in tmp_pair1_kb_d:
                        tmp_pair1_e_o_split = tmp_pair1_e_o.split(',')
                        tmp_pair1_e = tmp_pair1_e_o_split[0]
                        tmp_pair1_o = tmp_pair1_e_o_split[2].lstrip()
                        if tmp_pair1_e in w2v_model.key_to_index and tmp_pair1_o in w2v_model.key_to_index:
                            sim_score = w2v_model.similarity(tmp_pair1_e,tmp_pair1_o)
                            sim_socre2 = w2v_model.similarity(entity,tmp_pair1_o)
                            if sim_score > threshold_one_hop_triples: # and sim_score2 > threshold_two_hop_triples:
                                one_hop_triples_id.append(dict_csk_triples[tmp_pair1_e_o])
    #                         elif sim_score > threshold_one_hop_triples or sim_score2 > threshold_two_hop_triples:
                            else:
                                tmp_one_two_triple.append(dict_csk_triples[tmp_pair1_e_o])
                                tmp_only_two.append(tmp_pair1_o)
                        else:
                            continue
    #                     id_two_hop = [i for n, i in enumerate(tmp_one_two_triple) if i not in tmp_one_two_triple[:n]]
                        one_two_triple.append(tmp_one_two_triple)
                        only_two.append(tmp_only_two)
                    tmp_one_two_triple = []
                    tmp_only_tow = []
            elif entity == tmp_pair[1] and tmp_pair[0] not in post_entities:
                tmp_id_one_hop.append(dict_csk_entities[tmp_pair[0]])
                one_hop_triples_id.append(dict_csk_triples[e_o])
                if tmp_pair[0] in kb_dict:
                    tmp_pair2_kb_d = kb_dict[tmp_pair[0]]
                    tmp_one_two_triple = []
                    for tmp_pair2_e_o in tmp_pair2_kb_d:
                        tmp_pair2_e_o_split = tmp_pair2_e_o.split(',')
                        tmp_pair2_e = tmp_pair2_e_o_split[0]
                        tmp_pair2_o = tmp_pair2_e_o_split[2].lstrip()
                        if tmp_pair2_e in w2v_model.key_to_index and tmp_pair2_o in w2v_model.key_to_index:
                            sim_score = w2v_model.similarity(tmp_pair2_e,tmp_pair2_o)
                            sim_socre2 = w2v_model.similarity(entity,tmp_pair2_o)
                            if sim_score > threshold_one_hop_triples: # and sim_score2 > threshold_two_hop_triples:
                                one_hop_triples_id.append(dict_csk_triples[tmp_pair2_e_o])
    #                         elif sim_score > threshold_one_hop_triples or sim_score2 > threshold_two_hop_triples:
                            else:
                                tmp_one_two_triple.append(dict_csk_triples[tmp_pair2_e_o])
                                tmp_only_two.append(tmp_pair2_o)
                        else:
                            continue
                        one_two_triple.append(tmp_one_two_triple)
                        only_two.append(tmp_only_two)
                    tmp_one_two_triple = []
                    tmp_only_tow = []
            else:
                continue

    # ☆☆☆☆☆☆☆☆☆☆☆☆ all_entities_one_hop ☆☆☆☆☆☆☆☆☆☆☆☆
    id_one_hop = [i for n, i in enumerate(tmp_id_one_hop) if i not in tmp_id_one_hop[:n]]

    # ☆☆☆☆☆☆☆☆☆☆☆☆ all_triples_one_hop ☆☆☆☆☆☆☆☆☆☆☆☆
    one_hop_triples_id = [i for n, i in enumerate(one_hop_triples_id) if i not in one_hop_triples_id[:n]]

    one_two_triple = [i for n, i in enumerate(one_two_triple) if i not in one_two_triple[:n]]
    # only_two = [i for n, i in enumerate(only_two) if i not in only_two[:n]]
    # new_only_two = []
    # [new_only_two.append(item) for item in only_two if item not in new_only_two]
    new_only_two = list(set(only_two[0]))
    new = []
    for j in one_two_triple:
        length = len(j)
        if length > 2:
            new.append(random.sample(j, 3))
        else:
            new.append(j)
    one_two_triple = new

    only_two_two = []
    for entity in new_only_two:
        only_two_two.append(dict_csk_entities[entity])
    # print(only_two_two)
    only_two_two_two = random.sample(only_two_two, 100)
#     print(only_two_two_two)  
#     print(one_hop_triples_id)
#     print(id_one_hop)

    # induce response based on the one hop id for match_response_index_one_hop
    expected_length_response = random.randrange(20,30)
    expected_response = [-1] * expected_length_response
    expected_respones2 = [-1] * expected_length_response
    possible_one_hop_index_range = list(range(3, expected_length_response))
    num_of_one_hop = random.randrange(4, 9)
    possible_hop_index = random.sample(possible_one_hop_index_range, k=num_of_one_hop)
    num_of_two_hop = random.randrange(1, 3)
    num_of_one_hop = num_of_one_hop-num_of_two_hop
    possible_two_index = random.sample(possible_hop_index, k = num_of_two_hop)
    print('before possible_hop_index',possible_hop_index)

    # split num_of_one_hop for two_hop
    for i in possible_two_index:
        try:
            possible_hop_index.remove(i)
        except ValueError:
            pass
    print('possible_hop_index',possible_hop_index)
    print('possible_two_index',possible_two_index)
    replacements_one_hop = random.sample(id_one_hop, num_of_one_hop)
    replacements_two_hop = random.sample(only_two_two_two, num_of_two_hop)

    # print('replacements',replacements)
    # print('before', expected_response)
    for index in range(len(possible_hop_index)):
        expected_response[possible_hop_index[index]] = replacements_one_hop[index]
    for index in range(len(possible_two_index)):
        expected_respones2[possible_two_index[index]] = replacements_two_hop[index]

    # induce response based on the two_hop_id for match_response_index_two_hop

        
    data_format = {
        "all_entities_one_hop": id_one_hop,
        "post_triples": post_triples,
        "post": res,
        "response": ['None'] * expected_length_response,
        "match_response_index_one_hop": expected_response, 
        "only_two": only_two_two_two,
        "match_response_index_only_two": expected_respones2,
        "all_triples_one_hop": one_hop_triples_id,
        "one_two_triple": one_two_triple
    }
    json_object = json.dumps(data_format)
    with open('data.json', 'w', encoding='utf-8') as outfile:
        outfile.write(json_object)
    return data_format

In [214]:
"I wear glasses and at the age of seventeen I would have been mortified to be seen in a photo wearing them"

'I wear glasses and at the age of seventeen I would have been mortified to be seen in a photo wearing them'

In [237]:
new_format = generate_response_format(post,csk_triples, csk_entities, kb_dict, dict_csk_entities, dict_csk_triples)
print(new_format)

post_entities ['wear', 'glasses', 'age', 'seventeen', 'photo', 'wearing']
before possible_hop_index [13, 15, 6, 16, 3, 7]
possible_hop_index [13, 15, 16, 3, 7]
possible_two_index [6]
{'all_entities_one_hop': [18368, 8372, 8890, 18367, 18371, 12276, 14928, 7547, 269, 16375, 14971, 16042, 13127, 20158, 7358, 7550, 1580, 21446, 20754, 14872, 14663, 34, 19707, 11573, 21485, 12606, 4607, 12964, 18580, 20717, 8920, 5501, 19844, 4828, 14061, 11782, 7366, 5685, 16331, 20599, 21789, 11008, 433, 1807, 14538, 12322, 5651, 6647, 5013, 1704, 21455, 12218, 2063, 13534, 2432, 5270, 20039, 1621, 6281, 8810, 5859, 3949, 13111, 2924, 21532, 20160, 14379, 14629, 235, 97, 6081, 10433, 16664, 6204, 1615, 4851, 13163, 1190, 17808, 19422, 10491, 15887, 3100, 16338, 14940, 6969, 7407, 766, 16033, 3249, 7257, 9482, 4953, 22422, 17688, 1882, 14660, 382, 7939, 1826, 9174, 963, 1056, 17902, 15127, 282, 5519, 7427, 20814, 8854], 'post_triples': [0, 1, 2, 0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0],